In [1]:
from glob import glob
import numpy as np
import csv
import pandas as pd
from astropy.table import Table
from astropy.io import fits
import os

In [2]:
#define function with inputs field, ID, subfield, zphot; will add more as more data is gathered
def Magnitude(flux):
    #note: had to change to abs(flux) to avoid error, not sure if this is okay
    return -2.5*np.log10(abs(flux)) + 25

In [3]:
def logmass(mass):
    return round(np.log10(mass), 6)

In [14]:
def Bio_gen(field, ID, subfield, catalog, eazy_catalog, griz_catalog, exp_times):
    # field - eithxer North or South
    # ID - id number of galaxy
    # subfield - ie GN1 or ERSPRIME
    # zphot - photmetric redshift
    
    text1 ='<html>\n\
<title>{0}-{1}</title>\n\
<meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1" />\n\
<style>\n'.format(subfield, ID)
    
    text2 = '\n\
table.table1 {font-family: arial;border-collapse: collapse;width: 780px;}\n\
table.table1 td, table.table1 th {border: 1px solid #ffffff;text-align: center;padding: 7px;}\n\
\n\
table.table2 {font-family: arial;border-collapse: collapse;width: 900px;}\n\
table.table2 td, table.table2 th {border: none;text-align: center;padding: 7px;}\n\
\n\
table.table3 {font-family: arial;border-collapse: collapse;width: 820px;}\n\
table.table3 td, table.table3 td {border: none;text-align: center;padding-top: 7px; padding-bottom: 0px;}\n\
\n\
img.eazy {width: 100%;}\n\
img.grizli {width: 100%;}\n\
img.twod {width: 100%;}\n\
img.line {height: 183px;}\n\
img.cutout {width: 780px;\n\
    margin-top: 10px;\n\
    margin-bottom: 10px;\n\
    display: block;\n\
    margin-left: auto;\n\
    margin-right: auto;}\n\
\n\
.header {\n\
  text-align: center;\n\
  color: white;\n\
  font-size: 20px;\n\
}\n\
\n\
</style>\n\
\n\
<div class="header">\n'
    
    #define cutout width and height, as defined from find_size_of_object_cutouts
    if field == 'North':
        cutoutwidth = 13.0
        cutoutheight = 6.0
        
    if field == 'South':
        cutoutwidth = 6.8
        cutoutheight = 6.0
    
    text6 ='<h1>{0}-{1}</h1>\n\
</div>\n\
\n\
<div style="width:842px; margin: auto;">\n\
    <div style="width:780px;margin: auto;border:none;"> \n\
        <p style="float:right; margin:0px; border: none;">*Each postage stamp is {2}" x {3}"</p>'.format(subfield, ID, cutoutwidth, cutoutheight)
    
    text4 = '\n <img class="cutout" src="../../../cutouts/{0}/{0}_{1}.png">'.format(subfield, ID)
    
    text5 = '\n <table align="center" class="table1">\n\
    <tr>\n\
        <th>Field</th>\n\
        <th>ID</th>\n\
        <th>z<sub>phot</sub></th>\n\
        <th>z<sub>grism</sub></th>\n\
        <th>m<sub>098</sub></th>\n\
        <th>m<sub>105</sub></th>\n\
        <th>m<sub>125</sub></th>\n\
        <th>log(M<sub>*</sub>/M<sub>&#9737</sub>)</th>\n\
        <th>ra (deg)</th>\n\
        <th>dec (deg)</th></tr>\n\
    <tr>\n'
    
    #{0} - subfield, {1} - ID, {2} - zphot, {7} - log(mass), {8} - RA, {9} - DEC
    text3='        <th>{0}</th>\n\
        <th>{1}</th>\n\
        <th>{2:.2f}</th>\n\
        <th>{3:.4f}</th>\n\
        <th>{4}</th>\n\
        <th>{5:.2f}</th>\n\
        <th>{6:.2f}</th>\n\
        <th>{7:.2f}</th>\n\
        <th>{8:.5f}</th>\n\
        <th>{9:.5f}</th>\n\
    </tr>\n\
</table>\n\
</div>\n\
\n\
<div style="width:900px; float: right;"> \n\
<body bgcolor="#000000" text="#ffffff">\n\
\n\
<br>\n\
\n\
<div style="width: 80px;">\n\
<table align="center" class="table2">\n\
    <tr>\n\
        <th>\n\
            <p> EAZY <br /> P(z) <br /> SED </p></th>\n\
        <th>\n\
            <img class="eazy" src="../../../eazy_plots/{0}_{1}_EZ.png"></th>\n\
    <tr>\n\
        <th><p>Grizli <br /> P(z) <br /> SED <br /> </p></th>\n\
        <th><img class="grizli" src="../../../all_png/{0}_{1}.full.png"></th>\n\
</table>\n\
</div>\n\
\n\
<table align="right" class="table3">\n\
    <tr>\n\
        <td>\n\
            G102 <br> EXP TIME - {10} s\n\
        </td>\n\
        <td>\n\
            G141 <br> EXP TIME - {11} s\n\
        </td>\n\
    </tr>\n\
</table>\n\
\n\
<table align="center" class="table2">\n\
    <tr>\n\
        <th><div><p> Grizli <br /> 2D Spectra </p></div></th>\n\
        <th><img class="twod" src="../../../all_png/{0}_{1}.stack.png"></th>\n\
    <tr>\n\
        <th><p> Grizli <br /> Line Map </p></th>\n\
        <th><img class="line" src="../../../all_png/{0}_{1}.line.png"></th>\n\
    </tr>\n\
</table>\n\
</body>\n\
</html>\n'.format(subfield, ID, 
                  eazy_catalog.query('id == {0}'.format(int(ID))).z_phot.values[0],
                  griz_catalog.query('ID == {0}'.format(int(ID))).z_50.values[0],'-',
                  Magnitude(catalog.query('id == {0}'.format(int(ID))).f_F105W.values[0]),
                  Magnitude(catalog.query('id == {0}'.format(int(ID))).f_F125W.values[0]),
                  logmass(eazy_catalog.query('id == {0}'.format(int(ID))).mass.values[0]),
                  catalog.query('id == {0}'.format(int(ID))).ra.values[0],
                  catalog.query('id == {0}'.format(int(ID))).dec.values[0], 
                  exp_times.query('id == {0}'.format(int(ID))).T_g102.values[0], 
                  exp_times.query('id == {0}'.format(int(ID))).T_g141.values[0])

    #return everything as an array (so np.savetxt can read it)
    return [text1 + text2 + text6+ text4 + text5 + text3]

In [15]:
#read tables from different catalogs
gsd_cat = Table.read('../catalogs/goodss_3dhst.v4.4.cat', format = 'ascii').to_pandas()
gsd_eazy = Table.read(fits.open('../catalogs/goodss_3dhst.v4.4.zout.fits'), format = 'fits').to_pandas()

gnd_cat = Table.read('../catalogs/goodsn_3dhst.v4.4.cat', format = 'ascii').to_pandas()
gnd_eazy = Table.read(fits.open('../catalogs/goodsn_3dhst.v4.4.zout.fits'), format = 'fits').to_pandas()

In [10]:
#make temp bio page!! (to make sure the code works before making all of them)
griz_catalogs = Table.read(fits.open('../grizli_v2.1_cats/GS2_lines_grizli.fits'), format = 'fits').to_pandas()
exp_times = Table.read('../exp_times/GS2_exptimes.dat', format = 'ascii').to_pandas()
bio_page=Bio_gen('South', 46846, 'GS2', gsd_cat, gsd_eazy, griz_catalogs, exp_times) 
np.savetxt('/Users/rosaliaobrien/Desktop/TESTPAGE.html',bio_page,fmt='%s')

In [16]:
field_names = ['GN1', 'GN2', 'GN3', 'GN4', 'GN5', 'GN7', 'ERSPRIME', 'GS1', 'GS2', 'GS3', 'GS4', 'GS5']

In [17]:
for i in field_names:
    
    #grizli catalogs don't have all the exposure times (that exp_times has) for GS4
    if i is 'GS4':
        griz_catalogs = Table.read(fits.open('../grizli_v2.1_cats/' + i + '_lines_grizli.fits'), format = 'fits').to_pandas()
        exp_times = Table.read('../exp_times/'+i+'_exptimes.dat', format = 'ascii').to_pandas()
        ids=griz_catalogs['ID']
            
    #know that exp_times has gid's we want, so use those when we can
    else:
        griz_catalogs = Table.read(fits.open('../grizli_v2.1_cats/' + i + '_lines_grizli.fits'), format = 'fits').to_pandas()
        exp_times = Table.read('../exp_times/'+i+'_exptimes.dat', format = 'ascii').to_pandas()
        ids=exp_times['id']
    
    if 'N' in i:
        F = 'North'
        C = gnd_cat
        E = gnd_eazy
    else:
        F = 'South'
        C = gsd_cat
        E = gsd_eazy
    
    for ii in ids:
        
        print('Processing '+i, ii, '...')
        #print(F,i,ii)
        bio_page=Bio_gen(F, ii, i, C, E, griz_catalogs, exp_times) 
        np.savetxt('/Users/rosaliaobrien/research/website/bio_pages/{0}_{1}.html'.format(i,ii), bio_page, fmt='%s')

Processing GN1 35831 ...
Processing GN1 35921 ...
Processing GN1 35933 ...
Processing GN1 35935 ...
Processing GN1 35952 ...
Processing GN1 35959 ...
Processing GN1 36067 ...
Processing GN1 36114 ...
Processing GN1 36158 ...
Processing GN1 36203 ...
Processing GN1 36243 ...
Processing GN1 36303 ...
Processing GN1 36382 ...
Processing GN1 36420 ...
Processing GN1 36435 ...
Processing GN1 36456 ...
Processing GN1 36514 ...
Processing GN1 36521 ...
Processing GN1 36552 ...
Processing GN1 36556 ...
Processing GN1 36577 ...
Processing GN1 36596 ...
Processing GN1 36613 ...
Processing GN1 36623 ...
Processing GN1 36651 ...
Processing GN1 36663 ...
Processing GN1 36681 ...
Processing GN1 36717 ...
Processing GN1 36733 ...
Processing GN1 36735 ...
Processing GN1 36736 ...
Processing GN1 36738 ...
Processing GN1 36761 ...
Processing GN1 36773 ...
Processing GN1 36783 ...
Processing GN1 36786 ...
Processing GN1 36788 ...
Processing GN1 36795 ...
Processing GN1 36808 ...
Processing GN1 36809 ...


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log10
  


Processing GN1 37291 ...
Processing GN1 37295 ...
Processing GN1 37296 ...
Processing GN1 37299 ...
Processing GN1 37304 ...
Processing GN1 37306 ...
Processing GN1 37307 ...
Processing GN1 37314 ...
Processing GN1 37316 ...
Processing GN1 37330 ...
Processing GN1 37340 ...
Processing GN1 37343 ...
Processing GN1 37347 ...
Processing GN1 37349 ...
Processing GN1 37358 ...
Processing GN1 37363 ...
Processing GN1 37366 ...
Processing GN1 37367 ...
Processing GN1 37371 ...
Processing GN1 37375 ...
Processing GN1 37382 ...
Processing GN1 37392 ...
Processing GN1 37395 ...
Processing GN1 37399 ...
Processing GN1 37402 ...
Processing GN1 37403 ...
Processing GN1 37406 ...
Processing GN1 37411 ...
Processing GN1 37413 ...
Processing GN1 37421 ...
Processing GN1 37422 ...
Processing GN1 37424 ...
Processing GN1 37435 ...
Processing GN1 37444 ...
Processing GN1 37451 ...
Processing GN1 37454 ...
Processing GN1 37458 ...
Processing GN1 37460 ...
Processing GN1 37462 ...
Processing GN1 37464 ...


Processing GN2 10926 ...
Processing GN2 11011 ...
Processing GN2 11020 ...
Processing GN2 11066 ...
Processing GN2 11115 ...
Processing GN2 11128 ...
Processing GN2 11152 ...
Processing GN2 11159 ...
Processing GN2 11160 ...
Processing GN2 11201 ...
Processing GN2 11228 ...
Processing GN2 11288 ...
Processing GN2 11327 ...
Processing GN2 11334 ...
Processing GN2 11339 ...
Processing GN2 11344 ...
Processing GN2 11346 ...
Processing GN2 11381 ...
Processing GN2 11393 ...
Processing GN2 11429 ...
Processing GN2 11431 ...
Processing GN2 11443 ...
Processing GN2 11487 ...
Processing GN2 11502 ...
Processing GN2 11514 ...
Processing GN2 11526 ...
Processing GN2 11542 ...
Processing GN2 11561 ...
Processing GN2 11743 ...
Processing GN2 11768 ...
Processing GN2 11784 ...
Processing GN2 11785 ...
Processing GN2 11800 ...
Processing GN2 11802 ...
Processing GN2 11814 ...
Processing GN2 11837 ...
Processing GN2 11849 ...
Processing GN2 11941 ...
Processing GN2 11946 ...
Processing GN2 11964 ...


Processing GN2 17408 ...
Processing GN2 17412 ...
Processing GN2 17426 ...
Processing GN2 17433 ...
Processing GN2 17436 ...
Processing GN2 17437 ...
Processing GN2 17477 ...
Processing GN2 17481 ...
Processing GN2 17489 ...
Processing GN2 17502 ...
Processing GN2 17515 ...
Processing GN2 17530 ...
Processing GN2 17553 ...
Processing GN2 17579 ...
Processing GN2 17589 ...
Processing GN2 17626 ...
Processing GN2 17635 ...
Processing GN2 17646 ...
Processing GN2 17667 ...
Processing GN2 17708 ...
Processing GN2 17712 ...
Processing GN2 17714 ...
Processing GN2 17727 ...
Processing GN2 17748 ...
Processing GN2 17760 ...
Processing GN2 17782 ...
Processing GN2 17792 ...
Processing GN2 17799 ...
Processing GN2 17802 ...
Processing GN2 17822 ...
Processing GN2 17829 ...
Processing GN2 17861 ...
Processing GN2 17862 ...
Processing GN2 17886 ...
Processing GN2 17891 ...
Processing GN2 17898 ...
Processing GN2 17940 ...
Processing GN2 17942 ...
Processing GN2 17963 ...
Processing GN2 17977 ...


...

Processing GS3 42548 ...
Processing GS3 42555 ...
Processing GS3 42593 ...
Processing GS3 42607 ...
Processing GS3 42689 ...
Processing GS3 42778 ...
Processing GS3 42788 ...
Processing GS3 42809 ...
Processing GS3 42820 ...
Processing GS3 42822 ...
Processing GS3 42866 ...
Processing GS3 42903 ...
Processing GS3 42913 ...
Processing GS3 42915 ...
Processing GS3 42930 ...
Processing GS3 42981 ...
Processing GS3 43007 ...
Processing GS3 43048 ...
Processing GS3 43126 ...
Processing GS3 43144 ...
Processing GS3 43210 ...
Processing GS3 43248 ...
Processing GS3 43262 ...
Processing GS3 43264 ...
Processing GS3 43266 ...
Processing GS3 43283 ...
Processing GS3 43342 ...
Processing GS3 43356 ...
Processing GS3 43401 ...
Processing GS3 43567 ...
Processing GS3 43572 ...
Processing GS3 43631 ...
Processing GS3 43638 ...
Processing GS3 43762 ...
Processing GS3 43809 ...
Processing GS3 43913 ...
Processing GS3 43926 ...
Processing GS3 43948 ...
Processing GS3 44020 ...
Processing GS3 44083 ...


Processing GS4 21624 ...
Processing GS4 21633 ...
Processing GS4 21662 ...
Processing GS4 21676 ...
Processing GS4 21690 ...
Processing GS4 21710 ...
Processing GS4 21740 ...
Processing GS4 21749 ...
Processing GS4 21756 ...
Processing GS4 21761 ...
Processing GS4 21774 ...
Processing GS4 21797 ...
Processing GS4 21799 ...
Processing GS4 21817 ...
Processing GS4 21823 ...
Processing GS4 21829 ...
Processing GS4 21843 ...
Processing GS4 21849 ...
Processing GS4 21880 ...
Processing GS4 21890 ...
Processing GS4 21909 ...
Processing GS4 21922 ...
Processing GS4 21940 ...
Processing GS4 21946 ...
Processing GS4 21978 ...
Processing GS4 21986 ...
Processing GS4 22005 ...
Processing GS4 22027 ...
Processing GS4 22045 ...
Processing GS4 22046 ...
Processing GS4 22057 ...
Processing GS4 22078 ...
Processing GS4 22082 ...
Processing GS4 22103 ...
Processing GS4 22113 ...
Processing GS4 22172 ...
Processing GS4 22211 ...
Processing GS4 22252 ...
Processing GS4 22276 ...
Processing GS4 22278 ...


Processing GS4 27629 ...
Processing GS4 27660 ...
Processing GS4 27706 ...
Processing GS4 27795 ...
Processing GS4 27801 ...
Processing GS4 27867 ...
Processing GS4 27881 ...
Processing GS4 27904 ...
Processing GS4 27919 ...
Processing GS4 27925 ...
Processing GS4 27940 ...
Processing GS4 27957 ...
Processing GS4 27965 ...
Processing GS4 27973 ...
Processing GS4 27983 ...
Processing GS4 28022 ...
Processing GS4 28037 ...
Processing GS4 28052 ...
Processing GS4 28069 ...
Processing GS4 28087 ...
Processing GS4 28111 ...
Processing GS4 28114 ...
Processing GS4 28116 ...
Processing GS4 28119 ...
Processing GS4 28139 ...
Processing GS4 28151 ...
Processing GS4 28152 ...
Processing GS4 28188 ...
Processing GS4 28190 ...
Processing GS4 28197 ...
Processing GS4 28207 ...
Processing GS4 28218 ...
Processing GS4 28267 ...
Processing GS4 28310 ...
Processing GS4 28353 ...
Processing GS4 28356 ...
Processing GS4 28362 ...
Processing GS4 28382 ...
Processing GS4 28422 ...
Processing GS4 28424 ...


Processing GS5 38362 ...
Processing GS5 38378 ...
Processing GS5 38383 ...
Processing GS5 38414 ...
Processing GS5 38436 ...
Processing GS5 38477 ...
Processing GS5 38490 ...
Processing GS5 38513 ...
Processing GS5 38537 ...
Processing GS5 38559 ...
Processing GS5 38616 ...
Processing GS5 38625 ...
Processing GS5 38626 ...
Processing GS5 38636 ...
Processing GS5 38645 ...
Processing GS5 38671 ...
Processing GS5 38684 ...
Processing GS5 38715 ...
Processing GS5 38718 ...
Processing GS5 38786 ...
Processing GS5 38811 ...
Processing GS5 38833 ...
Processing GS5 38843 ...
Processing GS5 38849 ...
Processing GS5 38873 ...
Processing GS5 38932 ...
Processing GS5 38977 ...
Processing GS5 38982 ...
Processing GS5 39007 ...
Processing GS5 39015 ...
Processing GS5 39023 ...
Processing GS5 39065 ...
Processing GS5 39097 ...
Processing GS5 39133 ...
Processing GS5 39138 ...
Processing GS5 39171 ...
Processing GS5 39176 ...
Processing GS5 39194 ...
Processing GS5 39231 ...
Processing GS5 39253 ...


Processing GS5 44997 ...
Processing GS5 45024 ...
Processing GS5 45032 ...
Processing GS5 45095 ...
Processing GS5 45171 ...
Processing GS5 45250 ...
Processing GS5 45313 ...
Processing GS5 45337 ...
Processing GS5 45357 ...
Processing GS5 45361 ...
Processing GS5 45402 ...
Processing GS5 45414 ...
Processing GS5 45472 ...
Processing GS5 45474 ...
Processing GS5 45492 ...
Processing GS5 45532 ...
Processing GS5 45592 ...
Processing GS5 45597 ...
Processing GS5 45647 ...
Processing GS5 45650 ...
Processing GS5 45657 ...
Processing GS5 45717 ...
Processing GS5 45738 ...
Processing GS5 45787 ...
Processing GS5 45788 ...
Processing GS5 45795 ...
Processing GS5 45802 ...
Processing GS5 45940 ...
Processing GS5 45972 ...
Processing GS5 46001 ...
Processing GS5 46064 ...
Processing GS5 46129 ...
Processing GS5 46186 ...
Processing GS5 46216 ...
Processing GS5 46231 ...
Processing GS5 46379 ...
Processing GS5 46394 ...
Processing GS5 46399 ...
Processing GS5 46432 ...
Processing GS5 46486 ...


In [18]:
print('DONE')

DONE
